In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from copy import copy
from matplotlib import cm, colors
import cv2



# Takes a 2D numpy array as input (i.e. a numpy representation of an image)
# Different from tree since the empty ratio is different. Could implement a generic
# solution down the line
def processNextHouseImage(image): 
    objectwidth = 100
    objectheight = 100
    xspacing = 116
    yspacing = 300
    xstart = 293
    ystart = 445
    xend = 1730
    yend = 1770
    powderthickness = 80
    objectCoordinates = [[x, x+objectwidth, y, y+objectheight] for y in reversed(range(
        ystart, yend, objectheight + yspacing)) for x in range(xstart, xend, xspacing + objectwidth)]
    # Rest NYI

# Returns a list of images of each object, with the background filtered out
def processNextTreeImage(image, mask):
    objectwidth = 83
    objectheight = 122
    xspacing = 133
    yspacing = 270
    xstart = 293
    ystart = 268
    xend = 1730
    yend = 1770

    # objectinfo = pd.read_csv('Parameters.csv', names=["Object", "P", "S", "H", "Porosity", "Label"])

    objectCoordinates = [[x, x+objectwidth, y, y+objectheight] for y in reversed(range(
        ystart, yend, objectheight + yspacing)) for x in range(xstart, xend, xspacing + objectwidth)]
    # coorddf = pd.DataFrame(objectCoordinates, columns=['xstart', 'xend', 'ystart', 'yend'])
    # objectinfo = coorddf.join(objectinfo)

    objects = np.full((len(objectCoordinates), objectheight, objectwidth), np.nan)

    for index, object in objectCoordinates.iterrows():
        objects[index] = image[object.ystart:object.yend, object.xstart:object.xend]

    for object in objects:
        object[~mask] = np.nan

    return objects

# Conducts preprocessing. OldObjectLayers is a list of objects, with each object being a 3d numpy array

class preprocessor:
    def __init__(self, type, noOfObjects):
        self.type = type
        self.accumulatedLayers = [None]  * noOfObjects
    def preprocess(newLayer):
        for index, object in enumerate(newObjectLayers):
        if type == 'scatter' or type == 'spatstat':
            object = np.copy(object)
            xs = np.array(object, copy=True, dtype=np.float32)
            accumulatedObjectLayers[index][layerIndex] = xs
        elif type == 'moran':
            # Todo: consider revising and not doing on a layer-by-layer basis?
            xs = np.array(object, copy=True, dtype=np.float32)
            avg = np.nanmean(xs)
            stddev = np.nanstd(xs)
            xs = (xs - avg) / stddev
            accumulatedObjectLayers[index][layerIndex] = xs    

def preprocess(newObjectLayers, accumulatedObjectLayers, layerIndex, type):
    # print(rtn.shape)
    for index, object in enumerate(newObjectLayers):
        if type == 'scatter' or type == 'spatstat':
            object = np.copy(object)
            xs = np.array(object, copy=True, dtype=np.float32)
            accumulatedObjectLayers[index][layerIndex] = xs
        elif type == 'moran':
            # Todo: consider revising and not doing on a layer-by-layer basis?
            xs = np.array(object, copy=True, dtype=np.float32)
            avg = np.nanmean(xs)
            stddev = np.nanstd(xs)
            xs = (xs - avg) / stddev
            accumulatedObjectLayers[index][layerIndex] = xs
    return accumulatedObjectLayers

def calculateOutliers(newObjectLayers, accumulatedObjectLayers, type, neighbourhoodSetting, windowSize):
    outlierValues = []
    index = 0
    # return objects
    for object in newObjectLayers:
        object = np.copy(object)
        z, y, x = object.shape
        # Step 1: calculate neighbourhood
        neighbourkernel = np.ones((neighbourhoodSetting, neighbourhoodSetting)) / neighbourhoodSetting**2
        flatNeighbourhood = np.array([cv2.filter2D(src=layer, ddepth=-1, kernel=neighbourkernel) for layer in object])
        neighbourhoodValues = np.array([
            np.sum(flatNeighbourhood[layerIndex-windowSize:layerIndex], axis=0)/windowSize
            for layerIndex in range(windowSize, z+1)
        ])
        # Step 2: calculate outlier
        offset = windowSize // 2
        endoffset = windowSize - offset - 1

        ys = neighbourhoodValues[0:z-windowSize+1]
        xs = object[offset:z-endoffset]
        filter = np.logical_and(np.isfinite(xs), np.isfinite(ys))

        # plt.imshow(xs[0])
        # plt.figure()
        # plt.imshow(xs[0])
        # plt.figure()
        # if(index == 58):
        #     plt.imshow(xs[0])
        #     plt.figure()
        #     plt.imshow(ys[0])
        #     plt.figure()
        #     plt.imshow(filter[0])
        #     plt.figure()
        #     print(len(np.unique(filter)))
        numberOfFilterValues = len(np.unique(filter))
        # print("filterlength is: ", numberOfFilterValues)
        # print("index is:", index)
        assert numberOfFilterValues == 2, f"Expected filter to have two values, got: {numberOfFilterValues}"
        if type == 'spatstat':
            outliers = xs - ys
            avg = np.mean(outliers[filter])
            std = np.std(outliers[filter])
            outliers = (outliers - avg) / std
            outlierValues.append(outliers)
        else:
            with warnings.catch_warnings():
                line = np.polyfit(xs[filter].flatten(), ys[filter].flatten(), 1)
                p = np.poly1d(line)
                outlierValues.append(p(xs) - ys)
            assert(xs.shape == p(ys).shape)
        assert(len(np.unique(outlierValues[index])) > 1)
        assert(len(np.unique(np.isfinite(outlierValues[index]))) == 2)
        index+=1
    return outlierValues




In [ ]:
# High level picture
# We can batch prepare

trainingdata = gettrainingdata()
mask = createMask(trainingdata)


on(message):
    nextImg = processNextTreeImage(message, mask)
    
